In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## codes to transfer the data frames to sqlite

In [15]:
import sqlite3
import sqlalchemy as sqa
from sqlalchemy import text

In [14]:
engine = sqa.create_engine("sqlite:///phonepluse.db")



In [2]:
ATI_df = pd.read_csv(r'D:\Phone pluse project\extracted_csv_files\AGG\Agg_tns_India_df.csv')
ATI_df.drop('Unnamed: 0',axis=1, inplace=True)

In [3]:
ATI_df

,Year,Quater,Transaction_type,Transaction_count,Transaction_amount
0,2018,1,Recharge & bill payments,72550406,1.447271e+10
1,2018,1,Peer-to-peer payments,46982705,1.472459e+11
2,2018,1,Merchant payments,5368669,4.656679e+09
3,2018,1,Financial Services,3762820,8.158531e+08
4,2018,1,Others,5761576,4.643217e+09
...,...,...,...,...,...
95,2022,4,Merchant payments,6565690145,3.673601e+12
96,2022,4,Peer-to-peer payments,4459859921,1.457887e+13
97,2022,4,Recharge & bill payments,991049281,6.497768e+11
98,2022,4,Financial Services,4623387,5.360528e+09


In [5]:
def formatINR(number):
    s, *d = str(number).partition(".") 
    r = ",".join([s[x-2:x] for x in range(-3, -len(s), -2)][::-1] + [s[-3:]])
    return "".join([r] + d)

In [5]:
ATI_df['Trans_count_cf'] = ATI_df['Transaction_count'].apply(formatINR)

In [6]:
ATI_df

,Year,Quater,Transaction_type,Transaction_count,Transaction_amount,Trans_count_cf
0,2018,1,Recharge & bill payments,72550406,1.447271e+10,"7,25,50,406"
1,2018,1,Peer-to-peer payments,46982705,1.472459e+11,"4,69,82,705"
2,2018,1,Merchant payments,5368669,4.656679e+09,"53,68,669"
3,2018,1,Financial Services,3762820,8.158531e+08,"37,62,820"
4,2018,1,Others,5761576,4.643217e+09,"57,61,576"
...,...,...,...,...,...,...
95,2022,4,Merchant payments,6565690145,3.673601e+12,"6,56,56,90,145"
96,2022,4,Peer-to-peer payments,4459859921,1.457887e+13,"4,45,98,59,921"
97,2022,4,Recharge & bill payments,991049281,6.497768e+11,"99,10,49,281"
98,2022,4,Financial Services,4623387,5.360528e+09,"46,23,387"


'abs'

In [6]:
def format_cash(amount):
    def truncate_float(number, places):
        return int(number * (10 ** places)) / 10 ** places

    if amount < 1e3:
        return amount

    if 1e3 <= amount < 1e5:
        return str(truncate_float((amount / 1e5) * 100, 2)) + " K"

    if 1e5 <= amount < 1e7:
        return str(truncate_float((amount / 1e7) * 100, 2)) + " L"

    if amount > 1e7:
        return str(truncate_float(amount / 1e7, 2)) + " Cr"

In [53]:
final_format(14472713558)

'1,447.27 Cr'

In [12]:
# def formatINR(number):
#     s, *d = str(number).partition(".") 
#     r = ",".join([s[x-2:x] for x in range(-3, -len(s), -2)][::-1] + [s[-3:]])
#     return "".join([r] + d)

In [7]:
def final_format(amount):
    x = format_cash(amount)
    n,d,v = str(x).partition('.')
    value = formatINR(int(n))
    final = value + d + v
    return final

In [10]:
ATI_df['Transaction_amount'] = ATI_df['Transaction_amount'].apply(int)

In [12]:
ATI_df['Trans_amount_cf'] = ATI_df['Transaction_amount'].apply(final_format)

In [13]:
ATI_df

,Year,Quater,Transaction_type,Transaction_count,Transaction_amount,Trans_count_cf,Trans_amount_cf
0,2018,1,Recharge & bill payments,72550406,14472713558,"7,25,50,406","1,447.27 Cr"
1,2018,1,Peer-to-peer payments,46982705,147245883542,"4,69,82,705","14,724.58 Cr"
2,2018,1,Merchant payments,5368669,4656678915,"53,68,669",465.66 Cr
3,2018,1,Financial Services,3762820,815853105,"37,62,820",81.58 Cr
4,2018,1,Others,5761576,4643217301,"57,61,576",464.32 Cr
...,...,...,...,...,...,...,...
95,2022,4,Merchant payments,6565690145,3673601044800,"6,56,56,90,145","3,67,360.1 Cr"
96,2022,4,Peer-to-peer payments,4459859921,14578874324207,"4,45,98,59,921","14,57,887.43 Cr"
97,2022,4,Recharge & bill payments,991049281,649776847580,"99,10,49,281","64,977.68 Cr"
98,2022,4,Financial Services,4623387,5360528394,"46,23,387",536.05 Cr


In [ ]:
#  creating the table in sqlit3 data base

In [77]:
ATI_df.to_sql("Agg_tns_India", con=engine, index=False, if_exists='append')


100

## Aggregate Transaction State

In [14]:
ATS_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\AGG\Agg_trans_state_df.csv")
ATS_df.drop('Unnamed: 0',axis=1, inplace=True)
ATS_df

,State,Year,Quater,Transaction_type,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04
4,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
3589,west-bengal,2022,4,Peer-to-peer payments,184380244,6.202220e+11
3590,west-bengal,2022,4,Merchant payments,171667404,1.408080e+11
3591,west-bengal,2022,4,Recharge & bill payments,48921147,2.602663e+10
3592,west-bengal,2022,4,Financial Services,268388,2.611229e+08


In [15]:
ATS_df['Trans_count_cf'] = ATS_df['Transaction_count'].apply(formatINR)
ATS_df['Transaction_amount'] = ATS_df['Transaction_amount'].apply(int)
ATS_df['Trans_amount_cf'] = ATS_df['Transaction_amount'].apply(final_format)

In [16]:
ATS_df

,State,Year,Quater,Transaction_type,Transaction_count,Transaction_amount,Trans_count_cf,Trans_amount_cf
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1845307,"4,200",18.45 L
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,12138655,"1,871",1.21 Cr
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,452507,298,4.52 L
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,10601,33,10.6 K
4,andaman-&-nicobar-islands,2018,1,Others,256,184689,256,1.84 L
...,...,...,...,...,...,...,...,...
3589,west-bengal,2022,4,Peer-to-peer payments,184380244,620222000000,"18,43,80,244","62,022.2 Cr"
3590,west-bengal,2022,4,Merchant payments,171667404,140808000000,"17,16,67,404","14,080.8 Cr"
3591,west-bengal,2022,4,Recharge & bill payments,48921147,26026633153,"4,89,21,147","2,602.66 Cr"
3592,west-bengal,2022,4,Financial Services,268388,261122865,"2,68,388",26.11 Cr


In [79]:
ATS_df.to_sql("Agg_tns_state", con=engine, index=False, if_exists='append')

3594

## Aggregate user India

In [17]:
AUI_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\AGG\Agg_user_india_df.csv")
AUI_df.drop('Unnamed: 0',axis=1, inplace=True)
AUI_df

,Year,Quater,User_Brand,No_of_Users,Percent
0,2018,1,Xiaomi,11926334,0.254413
1,2018,1,Samsung,9609401,0.204988
2,2018,1,Vivo,5894293,0.125737
3,2018,1,Oppo,4479351,0.095554
4,2018,1,Realme,2376866,0.050703
...,...,...,...,...,...
182,2022,1,OnePlus,7974546,0.021382
183,2022,1,Motorola,5610140,0.015043
184,2022,1,Huawei,4882972,0.013093
185,2022,1,Tecno,4550078,0.012200


In [22]:
AUI_df['No_of_Users_cf'] = AUI_df['No_of_Users'].apply(formatINR)
AUI_df['Percent'] = AUI_df['Percent'].apply(lambda x : round(x,2))
# AUI_df['Trans_amount_cf'] = AUI_df['Transaction_amount'].apply(final_format)

In [23]:
AUI_df

,Year,Quater,User_Brand,No_of_Users,Percent,No_of_Users_cf
0,2018,1,Xiaomi,11926334,0.25,"1,19,26,334"
1,2018,1,Samsung,9609401,0.20,"96,09,401"
2,2018,1,Vivo,5894293,0.13,"58,94,293"
3,2018,1,Oppo,4479351,0.10,"44,79,351"
4,2018,1,Realme,2376866,0.05,"23,76,866"
...,...,...,...,...,...,...
182,2022,1,OnePlus,7974546,0.02,"79,74,546"
183,2022,1,Motorola,5610140,0.02,"56,10,140"
184,2022,1,Huawei,4882972,0.01,"48,82,972"
185,2022,1,Tecno,4550078,0.01,"45,50,078"


In [80]:
AUI_df.to_sql("Agg_user_india", con=engine, index=False, if_exists='append')

187

## Aggregate user state

In [21]:
AUS_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\AGG\Agg_user_state_df.csv")
AUS_df.drop('Unnamed: 0',axis=1, inplace=True)
AUS_df

,State,Year,Quater,User_Brand,No_of_Users,Percent
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,0.247033
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,0.214392
2,andaman-&-nicobar-islands,2018,1,Vivo,982,0.145697
3,andaman-&-nicobar-islands,2018,1,Oppo,501,0.074332
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,west-bengal,2022,1,Lenovo,330017,0.015056
6728,west-bengal,2022,1,Infinix,284678,0.012987
6729,west-bengal,2022,1,Asus,280347,0.012790
6730,west-bengal,2022,1,Apple,277752,0.012671


In [24]:
AUS_df['No_of_Users_cf'] = AUS_df['No_of_Users'].apply(formatINR)
AUS_df['Percent'] = AUS_df['Percent'].apply(lambda x : round(x,2))
AUS_df

,State,Year,Quater,User_Brand,No_of_Users,Percent,No_of_Users_cf
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,0.25,"1,665"
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,0.21,"1,445"
2,andaman-&-nicobar-islands,2018,1,Vivo,982,0.15,982
3,andaman-&-nicobar-islands,2018,1,Oppo,501,0.07,501
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.05,332
...,...,...,...,...,...,...,...
6727,west-bengal,2022,1,Lenovo,330017,0.02,"3,30,017"
6728,west-bengal,2022,1,Infinix,284678,0.01,"2,84,678"
6729,west-bengal,2022,1,Asus,280347,0.01,"2,80,347"
6730,west-bengal,2022,1,Apple,277752,0.01,"2,77,752"


In [81]:
AUS_df.to_sql("Agg_user_state", con=engine, index=False, if_exists='append')

6732


## Aggregate registered users and appopens



In [11]:
AUI_RU_AO_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\AGG\AUI_RU_AO_df.csv")
# AUI_RU_AO_df.drop('Unnamed: 0',axis=1, inplace=True)
AUI_RU_AO_df

,Year,Quater,Registered_Users,App_Opens
0,2018,1,46877867,0
1,2018,2,63648311,0
2,2018,3,80010952,0
3,2018,4,102262055,0
4,2019,1,123432699,0
5,2019,2,141808226,920413791
6,2019,3,159293934,3448366103
7,2019,4,178279085,4301071954
8,2020,1,197575164,4768944662
9,2020,2,218996326,4357634272


In [12]:
AUI_RU_AO_df['Reg_users_cf'] = AUI_RU_AO_df['Registered_Users'].apply(formatINR)
AUI_RU_AO_df['App_opens_cf'] = AUI_RU_AO_df['App_Opens'].apply(formatINR)


In [13]:
AUI_RU_AO_df

,Year,Quater,Registered_Users,App_Opens,Reg_users_cf,App_opens_cf
0,2018,1,46877867,0,"4,68,77,867",0
1,2018,2,63648311,0,"6,36,48,311",0
2,2018,3,80010952,0,"8,00,10,952",0
3,2018,4,102262055,0,"10,22,62,055",0
4,2019,1,123432699,0,"12,34,32,699",0
5,2019,2,141808226,920413791,"14,18,08,226","92,04,13,791"
6,2019,3,159293934,3448366103,"15,92,93,934","3,44,83,66,103"
7,2019,4,178279085,4301071954,"17,82,79,085","4,30,10,71,954"
8,2020,1,197575164,4768944662,"19,75,75,164","4,76,89,44,662"
9,2020,2,218996326,4357634272,"21,89,96,326","4,35,76,34,272"


In [16]:
AUI_RU_AO_df.to_sql('AUI_RU_AO', con=engine, index=False, if_exists='append')

20

## Map transaction India

In [25]:
MTI_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\MAP\Map_trans_india_df.csv")
MTI_df.drop('Unnamed: 0',axis=1, inplace=True)
MTI_df

,State,Year,Quater,Transaction_count,Transaction_amount
0,puducherry,2018,1,104212,1.658260e+08
1,tamil nadu,2018,1,6726622,1.126156e+10
2,uttar pradesh,2018,1,12537805,1.393997e+10
3,madhya pradesh,2018,1,8025395,8.681603e+09
4,andhra pradesh,2018,1,9039585,1.199628e+10
...,...,...,...,...,...
715,jammu & kashmir,2022,4,27316158,4.431530e+10
716,goa,2022,4,17192674,2.492499e+10
717,arunachal pradesh,2022,4,4632709,9.684910e+09
718,delhi,2022,4,515203457,7.449150e+11


In [26]:
MTI_df['Trans_count_cf'] = MTI_df['Transaction_count'].apply(formatINR)
MTI_df['Transaction_amount'] = MTI_df['Transaction_amount'].apply(int)
MTI_df['Trans_amount_cf'] = MTI_df['Transaction_amount'].apply(final_format)
MTI_df

,State,Year,Quater,Transaction_count,Transaction_amount,Trans_count_cf,Trans_amount_cf
0,puducherry,2018,1,104212,165825971,"1,04,212",16.58 Cr
1,tamil nadu,2018,1,6726622,11261557697,"67,26,622","1,126.15 Cr"
2,uttar pradesh,2018,1,12537805,13939970923,"1,25,37,805","1,393.99 Cr"
3,madhya pradesh,2018,1,8025395,8681603403,"80,25,395",868.16 Cr
4,andhra pradesh,2018,1,9039585,11996276391,"90,39,585","1,199.62 Cr"
...,...,...,...,...,...,...,...
715,jammu & kashmir,2022,4,27316158,44315295915,"2,73,16,158","4,431.52 Cr"
716,goa,2022,4,17192674,24924986371,"1,71,92,674","2,492.49 Cr"
717,arunachal pradesh,2022,4,4632709,9684909697,"46,32,709",968.49 Cr
718,delhi,2022,4,515203457,744915016018,"51,52,03,457","74,491.5 Cr"


In [82]:
MTI_df.to_sql("Map_trans_india", con=engine, index=False, if_exists='append')

720

## Map Transaction State

In [83]:
MTS_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\MAP\Map_trans_state_df.csv")
MTS_df.drop('Unnamed: 0',axis=1, inplace=True)
MTS_df

,State,Year,Quater,District_name,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
14631,west-bengal,2022,4,nadia district,12690126,2.804568e+10
14632,west-bengal,2022,4,birbhum district,7617444,1.614650e+10
14633,west-bengal,2022,4,purba medinipur district,14484229,3.309949e+10
14634,west-bengal,2022,4,maldah district,12492746,2.721861e+10


In [84]:
MTS_df['Trans_count_cf'] = MTS_df['Transaction_count'].apply(formatINR)
MTS_df['Transaction_amount'] = MTS_df['Transaction_amount'].apply(int)
MTS_df['Trans_amount_cf'] = MTS_df['Transaction_amount'].apply(final_format)
MTS_df

,State,Year,Quater,District_name,Transaction_count,Transaction_amount,Trans_count_cf,Trans_amount_cf
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,931663,442,9.31 L
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,12560249,"5,688",1.25 Cr
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1139848,528,11.39 L
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1317863,825,13.17 L
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,23948235,"9,395",2.39 Cr
...,...,...,...,...,...,...,...,...
14631,west-bengal,2022,4,nadia district,12690126,28045682168,"1,26,90,126","2,804.56 Cr"
14632,west-bengal,2022,4,birbhum district,7617444,16146502753,"76,17,444","1,614.65 Cr"
14633,west-bengal,2022,4,purba medinipur district,14484229,33099490372,"1,44,84,229","3,309.94 Cr"
14634,west-bengal,2022,4,maldah district,12492746,27218606574,"1,24,92,746","2,721.86 Cr"


In [85]:
MTS_df.to_sql("Map_trans_state", con=engine, index=False, if_exists='append')

14636

## Map User India

In [36]:
MUI_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\MAP\Map_user_india_df.csv")
MUI_df.drop('Unnamed: 0',axis=1, inplace=True)
MUI_df

,State,Year,Quater,User_count,AppOpens
0,puducherry,2018,1,49318,0
1,tamil nadu,2018,1,2104754,0
2,uttar pradesh,2018,1,4694250,0
3,madhya pradesh,2018,1,2553603,0
4,andhra pradesh,2018,1,3336450,0
...,...,...,...,...,...
715,jammu & kashmir,2022,4,1793434,60205865
716,goa,2022,4,855202,43005212
717,arunachal pradesh,2022,4,390307,21775038
718,delhi,2022,4,15764149,401864749


In [37]:
MUI_df['User_count_cf'] = MUI_df['User_count'].apply(formatINR)
MUI_df['AppOpens_cf'] = MUI_df['AppOpens'].apply(formatINR)
MUI_df

,State,Year,Quater,User_count,AppOpens,User_count_cf,AppOpens_cf
0,puducherry,2018,1,49318,0,"49,318",0
1,tamil nadu,2018,1,2104754,0,"21,04,754",0
2,uttar pradesh,2018,1,4694250,0,"46,94,250",0
3,madhya pradesh,2018,1,2553603,0,"25,53,603",0
4,andhra pradesh,2018,1,3336450,0,"33,36,450",0
...,...,...,...,...,...,...,...
715,jammu & kashmir,2022,4,1793434,60205865,"17,93,434","6,02,05,865"
716,goa,2022,4,855202,43005212,"8,55,202","4,30,05,212"
717,arunachal pradesh,2022,4,390307,21775038,"3,90,307","2,17,75,038"
718,delhi,2022,4,15764149,401864749,"1,57,64,149","40,18,64,749"


In [86]:
MUI_df.to_sql("Map_user_india", con=engine, index=False, if_exists='append')

720

## Map user state

In [38]:
MUS_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\MAP\Map_user_state_df.csv")
MUS_df.drop('Unnamed: 0',axis=1, inplace=True)
MUS_df

,State,Year,Quater,District_name,User_count,AppOpens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
14635,west-bengal,2022,4,nadia district,1359420,33853990
14636,west-bengal,2022,4,birbhum district,855236,20950662
14637,west-bengal,2022,4,purba medinipur district,1346908,38278506
14638,west-bengal,2022,4,maldah district,954892,29023743


In [39]:
MUS_df['User_count_cf'] = MUS_df['User_count'].apply(formatINR)
MUS_df['AppOpens_cf'] = MUS_df['AppOpens'].apply(formatINR)
MUS_df

,State,Year,Quater,District_name,User_count,AppOpens,User_count_cf,AppOpens_cf
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0,"5,846",0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0,"8,143",0
...,...,...,...,...,...,...,...,...
14635,west-bengal,2022,4,nadia district,1359420,33853990,"13,59,420","3,38,53,990"
14636,west-bengal,2022,4,birbhum district,855236,20950662,"8,55,236","2,09,50,662"
14637,west-bengal,2022,4,purba medinipur district,1346908,38278506,"13,46,908","3,82,78,506"
14638,west-bengal,2022,4,maldah district,954892,29023743,"9,54,892","2,90,23,743"


In [87]:
MUS_df.to_sql("Map_user_state", con=engine, index=False, if_exists='append')

14640

## Top Transaction India state

In [46]:
TTI_S_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\TOP\Top_tran_india_state_df.csv")
TTI_S_df.drop('Unnamed: 0',axis=1, inplace=True)
TTI_S_df

,State,Year,Quater,Transaction_count,Transaction_amount
0,maharashtra,2018,1,16387034,2.171161e+10
1,uttar pradesh,2018,1,12537805,1.393997e+10
2,karnataka,2018,1,12016899,1.921790e+10
3,west bengal,2018,1,11710225,1.062598e+10
4,andhra pradesh,2018,1,9039585,1.199628e+10
...,...,...,...,...,...
195,uttar pradesh,2022,4,836369071,1.376379e+12
196,andhra pradesh,2022,4,690017077,1.424725e+12
197,bihar,2022,4,517084649,9.360121e+11
198,delhi,2022,4,515203457,7.449150e+11


In [47]:
TTI_S_df['Trans_count_cf'] = TTI_S_df['Transaction_count'].apply(formatINR)
TTI_S_df['Transaction_amount'] = TTI_S_df['Transaction_amount'].apply(int)
TTI_S_df['Trans_amount_cf'] = TTI_S_df['Transaction_amount'].apply(final_format)
TTI_S_df

,State,Year,Quater,Transaction_count,Transaction_amount,Trans_count_cf,Trans_amount_cf
0,maharashtra,2018,1,16387034,21711613257,"1,63,87,034","2,171.16 Cr"
1,uttar pradesh,2018,1,12537805,13939970923,"1,25,37,805","1,393.99 Cr"
2,karnataka,2018,1,12016899,19217895314,"1,20,16,899","1,921.78 Cr"
3,west bengal,2018,1,11710225,10625978552,"1,17,10,225","1,062.59 Cr"
4,andhra pradesh,2018,1,9039585,11996276391,"90,39,585","1,199.62 Cr"
...,...,...,...,...,...,...,...
195,uttar pradesh,2022,4,836369071,1376378802069,"83,63,69,071","1,37,637.88 Cr"
196,andhra pradesh,2022,4,690017077,1424725083041,"69,00,17,077","1,42,472.5 Cr"
197,bihar,2022,4,517084649,936012149674,"51,70,84,649","93,601.21 Cr"
198,delhi,2022,4,515203457,744915016018,"51,52,03,457","74,491.5 Cr"


In [88]:
TTI_S_df.to_sql("Top_tran_india_state", con=engine, index=False, if_exists='append')

200

## Top Transactions india District 

In [48]:
TTI_D_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\TOP\Top_tran_india_district_df.csv")
TTI_D_df.drop('Unnamed: 0',axis=1, inplace=True)
TTI_D_df

,Year,Quater,District_name,Transaction_count,Transaction_amount
0,2018,1,bengaluru urban,8306513,1.416275e+10
1,2018,1,pune,3842929,5.378982e+09
2,2018,1,central delhi,2997714,4.312145e+09
3,2018,1,mumbai,2671029,3.923117e+09
4,2018,1,chennai,2656230,4.281138e+09
...,...,...,...,...,...
195,2022,4,mumbai suburban,248827345,3.102782e+11
196,2022,4,chhindwara,245710341,3.506381e+11
197,2022,4,patna,217434851,3.523938e+11
198,2022,4,khordha,208260666,2.684798e+11


In [49]:
TTI_D_df['Trans_count_cf'] = TTI_D_df['Transaction_count'].apply(formatINR)
TTI_D_df['Transaction_amount'] = TTI_D_df['Transaction_amount'].apply(int)
TTI_D_df['Trans_amount_cf'] = TTI_D_df['Transaction_amount'].apply(final_format)
TTI_D_df

,Year,Quater,District_name,Transaction_count,Transaction_amount,Trans_count_cf,Trans_amount_cf
0,2018,1,bengaluru urban,8306513,14162750303,"83,06,513","1,416.27 Cr"
1,2018,1,pune,3842929,5378982326,"38,42,929",537.89 Cr
2,2018,1,central delhi,2997714,4312145216,"29,97,714",431.21 Cr
3,2018,1,mumbai,2671029,3923116793,"26,71,029",392.31 Cr
4,2018,1,chennai,2656230,4281137762,"26,56,230",428.11 Cr
...,...,...,...,...,...,...,...
195,2022,4,mumbai suburban,248827345,310278216439,"24,88,27,345","31,027.82 Cr"
196,2022,4,chhindwara,245710341,350638062974,"24,57,10,341","35,063.8 Cr"
197,2022,4,patna,217434851,352393762150,"21,74,34,851","35,239.37 Cr"
198,2022,4,khordha,208260666,268479766618,"20,82,60,666","26,847.97 Cr"


In [90]:
TTI_D_df.to_sql("Top_tran_india_distric", con=engine, index=False, if_exists='append')

200

## Top transaction india pincode

In [50]:
TTI_P_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\TOP\Top_tran_india_pincode_df.csv")
TTI_P_df.drop('Unnamed: 0',axis=1, inplace=True)
TTI_P_df

,Year,Quater,Pincode,Transaction_count,Transaction_amount
0,2018,1,560001,2917356,4.684262e+09
1,2018,1,110006,2636324,3.714139e+09
2,2018,1,400008,2227885,3.105209e+09
3,2018,1,600003,2010196,2.997847e+09
4,2018,1,731101,1764956,2.366771e+09
...,...,...,...,...,...
195,2022,4,480001,240053791,3.418610e+11
196,2022,4,400070,180508440,2.263202e+11
197,2022,4,800001,167166031,2.696525e+11
198,2022,4,751001,140210762,1.846507e+11


In [51]:
TTI_P_df['Trans_count_cf'] = TTI_P_df['Transaction_count'].apply(formatINR)
TTI_P_df['Transaction_amount'] = TTI_P_df['Transaction_amount'].apply(int)
TTI_P_df['Trans_amount_cf'] = TTI_P_df['Transaction_amount'].apply(final_format)
TTI_P_df

,Year,Quater,Pincode,Transaction_count,Transaction_amount,Trans_count_cf,Trans_amount_cf
0,2018,1,560001,2917356,4684262246,"29,17,356",468.42 Cr
1,2018,1,110006,2636324,3714139312,"26,36,324",371.41 Cr
2,2018,1,400008,2227885,3105208723,"22,27,885",310.52 Cr
3,2018,1,600003,2010196,2997847000,"20,10,196",299.78 Cr
4,2018,1,731101,1764956,2366771285,"17,64,956",236.67 Cr
...,...,...,...,...,...,...,...
195,2022,4,480001,240053791,341860974379,"24,00,53,791","34,186.09 Cr"
196,2022,4,400070,180508440,226320223720,"18,05,08,440","22,632.02 Cr"
197,2022,4,800001,167166031,269652534972,"16,71,66,031","26,965.25 Cr"
198,2022,4,751001,140210762,184650659638,"14,02,10,762","18,465.06 Cr"


In [91]:
TTI_P_df.to_sql("Top_tran_india_pincode", con=engine, index=False, if_exists='append')

200

## Top Transactions State district

In [53]:
TTS_D_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\TOP\Top_trans_state_district_df.csv")
TTS_D_df.drop('Unnamed: 0',axis=1, inplace=True)
TTS_D_df

,State,Year,Quater,District_name,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,south andaman,5688,1.256025e+07
1,andaman-&-nicobar-islands,2018,1,nicobars,528,1.139849e+06
2,andaman-&-nicobar-islands,2018,1,north and middle andaman,442,9.316631e+05
3,andaman-&-nicobar-islands,2018,2,south andaman,9395,2.394824e+07
4,andaman-&-nicobar-islands,2018,2,nicobars,1120,3.072437e+06
...,...,...,...,...,...,...
5915,west-bengal,2022,4,south twenty four parganas,15651650,3.373698e+10
5916,west-bengal,2022,4,purba medinipur,14484229,3.309949e+10
5917,west-bengal,2022,4,hooghly,13931352,2.755409e+10
5918,west-bengal,2022,4,howrah,13350090,2.793786e+10


In [54]:
TTS_D_df['Trans_count_cf'] = TTS_D_df['Transaction_count'].apply(formatINR)
TTS_D_df['Transaction_amount'] = TTS_D_df['Transaction_amount'].apply(int)
TTS_D_df['Trans_amount_cf'] = TTS_D_df['Transaction_amount'].apply(final_format)
TTS_D_df

,State,Year,Quater,District_name,Transaction_count,Transaction_amount,Trans_count_cf,Trans_amount_cf
0,andaman-&-nicobar-islands,2018,1,south andaman,5688,12560249,"5,688",1.25 Cr
1,andaman-&-nicobar-islands,2018,1,nicobars,528,1139848,528,11.39 L
2,andaman-&-nicobar-islands,2018,1,north and middle andaman,442,931663,442,9.31 L
3,andaman-&-nicobar-islands,2018,2,south andaman,9395,23948235,"9,395",2.39 Cr
4,andaman-&-nicobar-islands,2018,2,nicobars,1120,3072436,"1,120",30.72 L
...,...,...,...,...,...,...,...,...
5915,west-bengal,2022,4,south twenty four parganas,15651650,33736981975,"1,56,51,650","3,373.69 Cr"
5916,west-bengal,2022,4,purba medinipur,14484229,33099490372,"1,44,84,229","3,309.94 Cr"
5917,west-bengal,2022,4,hooghly,13931352,27554087254,"1,39,31,352","2,755.4 Cr"
5918,west-bengal,2022,4,howrah,13350090,27937855668,"1,33,50,090","2,793.78 Cr"


In [92]:
TTS_D_df.to_sql("Top_trans_state_district", con=engine, index=False, if_exists='append')

5920

## Top Transactions State pincode

In [55]:
TTS_P_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\TOP\Top_trans_state_pincode_df.csv")
TTS_P_df.drop('Unnamed: 0',axis=1, inplace=True)
TTS_P_df

,State,Year,Quater,Pincode,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,744101.0,1622,2.769298e+06
1,andaman-&-nicobar-islands,2018,1,744103.0,1223,2.238042e+06
2,andaman-&-nicobar-islands,2018,1,744102.0,969,3.519060e+06
3,andaman-&-nicobar-islands,2018,1,744105.0,685,1.298561e+06
4,andaman-&-nicobar-islands,2018,1,744104.0,340,1.039715e+06
...,...,...,...,...,...,...
7134,west-bengal,2022,4,722101.0,2900058,5.748321e+09
7135,west-bengal,2022,4,700135.0,2471048,3.527457e+09
7136,west-bengal,2022,4,732101.0,2407008,5.052109e+09
7137,west-bengal,2022,4,700091.0,2348447,2.176640e+09


In [56]:
TTS_P_df['Trans_count_cf'] = TTS_D_df['Transaction_count'].apply(formatINR)
TTS_P_df['Transaction_amount'] = TTS_D_df['Transaction_amount'].apply(int)
TTS_P_df['Trans_amount_cf'] = TTS_D_df['Transaction_amount'].apply(final_format)
TTS_P_df

,State,Year,Quater,Pincode,Transaction_count,Transaction_amount,Trans_count_cf,Trans_amount_cf
0,andaman-&-nicobar-islands,2018,1,744101.0,1622,12560249.0,"5,688",1.25 Cr
1,andaman-&-nicobar-islands,2018,1,744103.0,1223,1139848.0,528,11.39 L
2,andaman-&-nicobar-islands,2018,1,744102.0,969,931663.0,442,9.31 L
3,andaman-&-nicobar-islands,2018,1,744105.0,685,23948235.0,"9,395",2.39 Cr
4,andaman-&-nicobar-islands,2018,1,744104.0,340,3072436.0,"1,120",30.72 L
...,...,...,...,...,...,...,...,...
7134,west-bengal,2022,4,722101.0,2900058,NaN,NaN,NaN
7135,west-bengal,2022,4,700135.0,2471048,NaN,NaN,NaN
7136,west-bengal,2022,4,732101.0,2407008,NaN,NaN,NaN
7137,west-bengal,2022,4,700091.0,2348447,NaN,NaN,NaN


In [93]:
TTS_P_df.to_sql("Top_trans_state_pincode", con=engine, index=False, if_exists='append')

7139

## Top Users India state

In [57]:
TUI_S_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\TOP\Top_user_india_state_df.csv")
TUI_S_df.drop('Unnamed: 0',axis=1, inplace=True)
TUI_S_df

,State,Year,Quater,User_count
0,maharashtra,2018,1,6106994
1,uttar pradesh,2018,1,4694250
2,karnataka,2018,1,3717763
3,andhra pradesh,2018,1,3336450
4,telangana,2018,1,3315560
...,...,...,...,...
195,telangana,2022,4,25656381
196,west bengal,2022,4,25536381
197,tamil nadu,2022,4,25136112
198,madhya pradesh,2022,4,22913924


In [61]:
TUI_S_df['User_count_cf'] = TUI_S_df['User_count'].apply(formatINR)
TUI_S_df

,State,Year,Quater,User_count,User_count_cf
0,maharashtra,2018,1,6106994,"61,06,994"
1,uttar pradesh,2018,1,4694250,"46,94,250"
2,karnataka,2018,1,3717763,"37,17,763"
3,andhra pradesh,2018,1,3336450,"33,36,450"
4,telangana,2018,1,3315560,"33,15,560"
...,...,...,...,...,...
195,telangana,2022,4,25656381,"2,56,56,381"
196,west bengal,2022,4,25536381,"2,55,36,381"
197,tamil nadu,2022,4,25136112,"2,51,36,112"
198,madhya pradesh,2022,4,22913924,"2,29,13,924"


In [94]:
TUI_S_df.to_sql("Top_user_india_state", con=engine, index=False, if_exists='append')

200

## Top user india district

In [63]:
TUI_D_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\TOP\Top_user_india_districts_df.csv")
TUI_D_df.drop('Unnamed: 0',axis=1, inplace=True)
TUI_D_df

,Year,Quater,District_name,User_count
0,2018,1,bengaluru urban,1922368
1,2018,1,pune,1211643
2,2018,1,jaipur,900773
3,2018,1,mumbai suburban,719300
4,2018,1,hyderabad,655175
...,...,...,...,...
195,2022,4,hyderabad,4962214
196,2022,4,ahmadabad,4530853
197,2022,4,rangareddy,4296707
198,2022,4,north twenty four parganas,3755017


In [64]:
TUI_D_df['User_count_cf'] = TUI_D_df['User_count'].apply(formatINR)
TUI_D_df

,Year,Quater,District_name,User_count,User_count_cf
0,2018,1,bengaluru urban,1922368,"19,22,368"
1,2018,1,pune,1211643,"12,11,643"
2,2018,1,jaipur,900773,"9,00,773"
3,2018,1,mumbai suburban,719300,"7,19,300"
4,2018,1,hyderabad,655175,"6,55,175"
...,...,...,...,...,...
195,2022,4,hyderabad,4962214,"49,62,214"
196,2022,4,ahmadabad,4530853,"45,30,853"
197,2022,4,rangareddy,4296707,"42,96,707"
198,2022,4,north twenty four parganas,3755017,"37,55,017"


In [95]:
TUI_D_df.to_sql("Top_user_india_district", con=engine, index=False, if_exists='append')

200

## Top user india pincode

In [66]:
TUI_P_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\TOP\Top_user_india_pincodes_df.csv")
TUI_P_df.drop('Unnamed: 0',axis=1, inplace=True)
TUI_P_df

,Year,Quater,Pincode,User_count
0,2018,1,201301,114625
1,2018,1,500072,105012
2,2018,1,560068,98487
3,2018,1,110059,95496
4,2018,1,110092,83600
...,...,...,...,...
195,2022,4,122001,542209
196,2022,4,110092,534009
197,2022,4,110086,477514
198,2022,4,121004,468607


In [67]:
TUI_P_df['User_count_cf'] = TUI_D_df['User_count'].apply(formatINR)
TUI_P_df

,Year,Quater,Pincode,User_count,User_count_cf
0,2018,1,201301,114625,"19,22,368"
1,2018,1,500072,105012,"12,11,643"
2,2018,1,560068,98487,"9,00,773"
3,2018,1,110059,95496,"7,19,300"
4,2018,1,110092,83600,"6,55,175"
...,...,...,...,...,...
195,2022,4,122001,542209,"49,62,214"
196,2022,4,110092,534009,"45,30,853"
197,2022,4,110086,477514,"42,96,707"
198,2022,4,121004,468607,"37,55,017"


In [96]:
TUI_P_df.to_sql("Top_user_india_pincode", con=engine, index=False, if_exists='append')

200

## Top User State District

In [68]:
TUS_D_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\TOP\Top_users_state_district_df.csv")
TUS_D_df.drop('Unnamed: 0',axis=1, inplace=True)
TUS_D_df

,State,Year,Quater,District_name,User_count
0,andaman-&-nicobar-islands,2018,1,south andaman,5846
1,andaman-&-nicobar-islands,2018,2,south andaman,8143
2,andaman-&-nicobar-islands,2018,3,south andaman,10474
3,andaman-&-nicobar-islands,2018,4,south andaman,13133
4,andaman-&-nicobar-islands,2019,1,south andaman,15961
...,...,...,...,...,...
715,west-bengal,2021,4,north twenty four parganas,3043008
716,west-bengal,2022,1,north twenty four parganas,3224459
717,west-bengal,2022,2,north twenty four parganas,3397791
718,west-bengal,2022,3,north twenty four parganas,3578500


In [69]:
TUS_D_df['User_count_cf'] = TUS_D_df['User_count'].apply(formatINR)
TUS_D_df

,State,Year,Quater,District_name,User_count,User_count_cf
0,andaman-&-nicobar-islands,2018,1,south andaman,5846,"5,846"
1,andaman-&-nicobar-islands,2018,2,south andaman,8143,"8,143"
2,andaman-&-nicobar-islands,2018,3,south andaman,10474,"10,474"
3,andaman-&-nicobar-islands,2018,4,south andaman,13133,"13,133"
4,andaman-&-nicobar-islands,2019,1,south andaman,15961,"15,961"
...,...,...,...,...,...,...
715,west-bengal,2021,4,north twenty four parganas,3043008,"30,43,008"
716,west-bengal,2022,1,north twenty four parganas,3224459,"32,24,459"
717,west-bengal,2022,2,north twenty four parganas,3397791,"33,97,791"
718,west-bengal,2022,3,north twenty four parganas,3578500,"35,78,500"


In [97]:
TUS_D_df.to_sql("Top_users_state_district", con=engine, index=False, if_exists='append')

720

## Top user state pincode

In [70]:
TUS_P_df = pd.read_csv(r"D:\Phone pluse project\extracted_csv_files\TOP\Top_users_state_pincodes_df.csv")
TUS_P_df.drop('Unnamed: 0',axis=1, inplace=True)
TUS_P_df

,State,Year,Quater,Pincode,User_count
0,andaman-&-nicobar-islands,2018,1,744103,1608
1,andaman-&-nicobar-islands,2018,2,744103,2188
2,andaman-&-nicobar-islands,2018,3,744103,2741
3,andaman-&-nicobar-islands,2018,4,744103,3373
4,andaman-&-nicobar-islands,2019,1,744103,4136
...,...,...,...,...,...
715,west-bengal,2021,4,711101,132051
716,west-bengal,2022,1,711101,139738
717,west-bengal,2022,2,711101,147152
718,west-bengal,2022,3,711101,154935


In [71]:
TUS_P_df['User_count_cf'] = TUS_D_df['User_count'].apply(formatINR)
TUS_P_df

,State,Year,Quater,Pincode,User_count,User_count_cf
0,andaman-&-nicobar-islands,2018,1,744103,1608,"5,846"
1,andaman-&-nicobar-islands,2018,2,744103,2188,"8,143"
2,andaman-&-nicobar-islands,2018,3,744103,2741,"10,474"
3,andaman-&-nicobar-islands,2018,4,744103,3373,"13,133"
4,andaman-&-nicobar-islands,2019,1,744103,4136,"15,961"
...,...,...,...,...,...,...
715,west-bengal,2021,4,711101,132051,"30,43,008"
716,west-bengal,2022,1,711101,139738,"32,24,459"
717,west-bengal,2022,2,711101,147152,"33,97,791"
718,west-bengal,2022,3,711101,154935,"35,78,500"


In [98]:
TUS_P_df.to_sql("Top_users_state_pincode", con=engine, index=False, if_exists='append')

720